---
sidebar_label: PolarisAIDataInsight
---

# PolarisAIDataInsightLoader

- TODO: Make sure API reference link is correct.

This notebook provides a quick overview for getting started with PolarisAIDataInsight [document loader](https://python.langchain.com/docs/concepts/document_loaders). For detailed documentation of all PolarisAIDataInsightLoader features and configurations head to the [API reference](https://python.langchain.com/v0.2/api_reference/community/document_loaders/langchain_community.document_loaders.langchain_polarisoffice_datainsight_loader.PolarisAIDataInsightLoader.html).

## Overview
### Integration details

- TODO: Fill in table features.
- TODO: Remove JS support link if not relevant, otherwise ensure link is correct.
- TODO: Make sure API reference links are correct.

| Class | Package | Local | Serializable | [JS support](###TODO)|
| :--- | :--- | :---: | :---: |  :---: |
| [PolarisAIDataInsightLoader](###TODO) | [langchain-polaris-ai-datainsight](###TODO) | ❌ | ❌ | ✅ | 
### Loader features
| Source | Document Lazy Loading | Native Async Support
| :---: | :---: | :---: | 
| PolarisAIDataInsightLoader | ✅ | ❌ | 

## Setup

To access PolarisAIDataInsight document loader you'll need to install the `langchain-polaris-ai-datainsight` integration package, and create a **Polaris AI DataInsight** account and get an API key.

### Credentials

Head to [here](https://datainsight.polarisoffice.com/documentation/quickstart) to sign up to PolarisAIDataInsight and generate an API key. Once you've done this set the POLARIS_AI_DATA_INSIGHT_API_KEY environment variable:

In [ ]:
import getpass
import os

os.environ["POLARIS_AI_DATA_INSIGHT_API_KEY"] = getpass.getpass("Enter your PolarisAIDataInsight API key: ")

### Installation

Install **langchain-polaris-ai-datainsight**.

In [ ]:
%pip install -qU langchain-polaris-ai-datainsight

## Initialization

Now we can instantiate our model object and load documents:

In [ ]:
from langchain_polaris_ai_datainsight import PolarisAIDataInsightLoader

loader = PolarisAIDataInsightLoader(
    file_path="/path/to/file",
    mode="page"     # "element", "page", or "single". (default is "single") 
)

## Load

In [ ]:
docs = loader.load()
docs[0]

In [ ]:
print(docs[0].metadata)

## Lazy Load

In [ ]:
page = []
for doc in loader.lazy_load():
    page.append(doc)

print(page[0].metadata)